# Import libraries

In [ ]:
from datetime import datetime

In [ ]:
import numpy as np
from numpy import log

import pandas as pd
from pandas.api.types import CategoricalDtype, is_numeric_dtype

import geopandas as gpd

from sklearn import preprocessing

/usr/local/lib/python3.6/dist-packages/geopandas/_compat.py:88: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import ticker
%matplotlib

plt.rcParams["figure.figsize"] = (12, 8)
# For visualising data on interactive maps. Supports Image, Video, GeoJSON, TopoJSON overlays
import folium 
from branca.colormap import LinearColormap

from bokeh.plotting import figure, show
from bokeh.tile_providers import get_provider, Vendors # tile provider is the underlying map
from bokeh.io import reset_output, output_notebook, save # to display plots inside Jupyter

reset_output()
output_notebook()

Using matplotlib backend: agg


Loading BokehJS ...

# Extract, Load, Transform

In [ ]:
ZONES = pd.read_feather('./Data/TaxiZones.feather')
ZONES.tail()

,LocationID,Borough,Zone,service_zone
260,261,Manhattan,World Trade Center,Yellow Zone
261,262,Manhattan,Yorkville East,Yellow Zone
262,263,Manhattan,Yorkville West,Yellow Zone
263,264,Unknown,NV,None
264,265,Unknown,None,None


In [1]:
FILE_PATH = f"./Data/$LICENSE$_$YEAR$/$LICENSE$_tripdata_$YEAR$-$MONTH$.feather" 
MONTHS = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

In [ ]:
def feather_to_df(license, year, month):
    file_path = FILE_PATH.replace("$YEAR$", year).replace("$MONTH$", month).replace("$LICENSE$", license)
    print('Reading from: '+file_path)
    df = pd.read_feather(file_path).drop('index', axis=1)
    df.name = f"{license}_{year}_{MONTHS[int(month)-1]}"
    return df

In [ ]:
def logify(x): 
    return log(x) if x else 0

In [ ]:
def logify_prop(x):
    return log(x+1) if x else 0

In [ ]:
def logit(p):
    return log(p/(1-p))

In [ ]:
MONTH_CATS = CategoricalDtype(MONTHS, ordered=True)
COMPANY_CATS = CategoricalDtype(['Yellow', 'Green', 'Uber', 'Juno', 'Via', 'Lyft'], ordered=False)
vendors = {
            1: 'Creative Mobile Technologies, LLC.',
            2: 'Verifone Inc.',
            4: 'Flywheel Inc.'
}
VENDOR_CATS = CategoricalDtype(vendors.values() , ordered=False)
ratecodes = {
            1: 'Standard', 
            2: 'JFK', 
            3: 'Newark', 
            4: 'Nassau/Westchester', 
            5: 'Negotiated', 
            6: 'Group', 
            99: 'Unknown'
        }
RATE_CATS = CategoricalDtype(ratecodes.values(), ordered=False)
payment_types = {
#            0: 'Unknown',
            1: 'Credit Card',
            2: 'Cash',
            3: 'No charge', 
            4: 'Dispute', 
            5: 'Unknown', 
            6: 'Voided Trip'
        }
PAYMENT_CATS = CategoricalDtype(payment_types.values(), ordered=False)
company_license_codes = {
                    'HV0003': 'Uber',
                    'HV0005': 'Lyft',
                    'HV0002': 'Juno',
                    'HV0004': 'Via'
                }
COMPANY_CATS = CategoricalDtype(list(company_license_codes.values()) +['Yellow', 'Green'], ordered=False)
license_types = {
    'yellow': 'Medallion',
    'green': 'Street Hail Liveries',
    'fhv': 'High-Volume For-Hire Services',
    'fhvhv': 'High-Volume For-Hire Services'
}
LICENSE_CATS = CategoricalDtype(set(license_types.values()))
LOCATIONID_CATS = CategoricalDtype(range(1, 257))

In [ ]:
def cast_green_yellow(df):
    
    df['company'] = 'Yellow' if 'yellow' in df.name else 'Green'
    df.company = df.company.astype(COMPANY_CATS)
                
    df.VendorID = df.VendorID.astype('category').cat.rename_categories(vendors).astype(VENDOR_CATS)
    df.passenger_count = df.passenger_count.replace(to_replace=0, value=np.nan).astype('Int64')
    df.trip_distance = df.trip_distance.astype('float')
    df[['PULocationID', 'DOLocationID']] = df[['PULocationID', 'DOLocationID']].replace([264, 265], np.nan).astype('Int64').astype(LOCATION_CATS)
    df.mta_tax = df.mta_tax.astype('float')
    df.store_and_fwd_flag = df.store_and_fwd_flag.fillna(False).astype('bool')
    df.improvement_surcharge = df.improvement_surcharge.astype('float')
    df.fare_amount = df.fare_amount.astype('float')
    
    df['license_type'] = 'Medallion' if 'yellow' in df.name else 'Street Hail Liveries'
    df.license_type = df.license_type.astype(LICENSE_CATS)
    
    tl = 't' if 'yellow' in df.name else 'l'
    time_cols = [f'{tl}pep_pickup_datetime', f'{tl}pep_dropoff_datetime']
    
    for time_col in time_cols:    
        df[time_col] = pd.to_datetime(df[time_col], yearfirst=True, errors='raise')
        
    df = df.rename(columns={
        'tpep_pickup_datetime': 'pickup_datetime',
        'lpep_pickup_datetime': 'pickup_datetime',
        'tpep_dropoff_datetime': 'dropoff_datetime',
        'lpep_dropoff_datetime': 'dropoff_datetime',
    })
    df['trip_duration'] = df.dropoff_datetime - df.pickup_datetime

    df.RatecodeID = df.RatecodeID.fillna(99).astype('category').cat.rename_categories(ratecodes).astype(RATE_CATS)
    df = df.rename(columns={'RatecodeID': 'ratecode', 'VendorID': 'vendor'})


    df.payment_type = df.payment_type.replace(to_replace=5, value=np.nan).astype('Int64').astype('category').cat.rename_categories(payment_types).astype(PAYMENT_CATS)
    
    return df

In [ ]:
def cast_fhv(df):
    
    if 'hvfhs_license_num' in df.columns:
        df.hvfhs_license_num = (df.hvfhs_license_num
                                .astype('category')
                                .cat.rename_categories(company_license_codes)
                                .astype(COMPANY_CATS)
        )
        df = df.rename(columns={'hvfhs_license_num': 'company'})

    df.dispatching_base_num = df.dispatching_base_num.astype('category')
    
    df.SR_Flag = df.SR_Flag.fillna(0).astype('bool')
    df.pickup_datetime = pd.to_datetime(df.pickup_datetime, yearfirst=True)
    df.dropoff_datetime = pd.to_datetime(df.dropoff_datetime, yearfirst=True)
    df[['PULocationID', 'DOLocationID']] = df[['PULocationID', 'DOLocationID']].replace([264, 265], np.nan).astype('Int64').astype(LOCATION_CATS)
    
    return df


In [ ]:
def ETL(license, year, start, end):

    dfs = dict()

    for i, month in list(enumerate(MONTHS))[start-1: end]:

        month_num = i + 1
        month_str = f"{month_num:02}"
        
        print('\n\n' + license + year + '-' + month_str)
        print('\t reading feather...')
        df = feather_to_df(license, year, month_str)    
        
        if license in ('green', 'yellow'):
            df = cast_green_yellow(df)
        elif license in ('fhv', 'fhvhv'):
            df = cast_fhv(df)
        dfs[month] = df
    
    grand_df = (pd.concat(dfs.values(), keys=dfs.keys(), axis=0)
                .reset_index()
                .drop('level_1', axis=1)
                .rename({'level_0':'month'}, axis=1))
    
    grand_df.month = grand_df.month.astype(MONTH_CATS)
    grand_df[['PULocationID', 'DOLocationID']] = grand_df[['PULocationID', 'DOLocationID']].astype('category')
    if 'fhv' in license:
        grand_df.dispatching_base_num = grand_df.dispatching_base_num.astype('category') 
        
    grand_df.name = f"{license}_{year}"
    
    return grand_df

In [ ]:
yellow_19_df = ETL('yellow', '2019', 1, 12)

In [ ]:
green_19_df = ETL('green', '2019', 1, 12)

In [ ]:
fhvhv_19_df = ETL('fhvhv', '2019', 2, 12)

In [ ]:
fhv_19_df = ETL('fhv', '2019', 1, 1)

## This is everything loaded

In [ ]:
yellow_19_df

,index,month,vendor,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,ratecode,store_and_fwd_flag,PULocationID,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,company,license_type,trip_duration
0,0,Jan,"Creative Mobile Technologies, LLC.",2019-01-01 00:46:40,2019-01-01 00:53:20,1.0,1.50,Standard,True,151,...,0.50,0.5,1.65,0.00,0.3,9.95,NaN,Yellow,Medallion,0 days 00:06:40
1,1,Jan,"Creative Mobile Technologies, LLC.",2019-01-01 00:59:47,2019-01-01 01:18:59,1.0,2.60,Standard,True,239,...,0.50,0.5,1.00,0.00,0.3,16.30,NaN,Yellow,Medallion,0 days 00:19:12
2,2,Jan,Verifone Inc.,2018-12-21 13:48:30,2018-12-21 13:52:40,3.0,0.00,Standard,True,236,...,0.50,0.5,0.00,0.00,0.3,5.80,NaN,Yellow,Medallion,0 days 00:04:10
3,3,Jan,Verifone Inc.,2018-11-28 15:52:25,2018-11-28 15:55:45,5.0,0.00,Standard,True,193,...,0.50,0.5,0.00,0.00,0.3,7.55,NaN,Yellow,Medallion,0 days 00:03:20
4,4,Jan,Verifone Inc.,2018-11-28 15:56:57,2018-11-28 15:58:33,5.0,0.00,JFK,True,193,...,0.00,0.5,0.00,0.00,0.3,55.55,NaN,Yellow,Medallion,0 days 00:01:36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84399013,84399013,Dec,NaN,2019-12-31 00:10:00,2019-12-31 00:59:00,NaN,18.99,Unknown,False,181,...,2.75,0.5,0.00,0.00,0.3,68.10,0.0,Yellow,Medallion,0 days 00:49:00
84399014,84399014,Dec,NaN,2019-12-31 00:07:00,2019-12-31 00:46:00,NaN,12.78,Unknown,False,230,...,2.75,0.5,0.00,6.12,0.3,41.99,0.0,Yellow,Medallion,0 days 00:39:00
84399015,84399015,Dec,NaN,2019-12-31 00:20:00,2019-12-31 00:47:00,NaN,18.52,Unknown,False,219,...,2.75,0.5,0.00,6.12,0.3,61.30,0.0,Yellow,Medallion,0 days 00:27:00
84399016,84399016,Dec,NaN,2019-12-31 00:50:00,2019-12-31 01:21:00,NaN,13.13,Unknown,False,161,...,2.75,0.5,0.00,6.12,0.3,47.69,0.0,Yellow,Medallion,0 days 00:31:00


In [ ]:
green_19_df

,index,month,vendor,pickup_datetime,dropoff_datetime,store_and_fwd_flag,ratecode,PULocationID,DOLocationID,passenger_count,...,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,company,license_type,trip_duration
0,0,Jan,Verifone Inc.,2018-12-21 15:17:29,2018-12-21 15:18:57,True,Standard,NaN,NaN,5.0,...,0.0,NaN,0.3,4.30,Cash,1.0,NaN,Green,Street Hail Liveries,0 days 00:01:28
1,1,Jan,Verifone Inc.,2019-01-01 00:10:16,2019-01-01 00:16:32,True,Standard,97,49,2.0,...,0.0,NaN,0.3,7.30,Cash,1.0,NaN,Green,Street Hail Liveries,0 days 00:06:16
2,2,Jan,Verifone Inc.,2019-01-01 00:27:11,2019-01-01 00:31:38,True,Standard,49,189,2.0,...,0.0,NaN,0.3,5.80,Credit Card,1.0,NaN,Green,Street Hail Liveries,0 days 00:04:27
3,3,Jan,Verifone Inc.,2019-01-01 00:46:20,2019-01-01 01:04:54,True,Standard,189,17,2.0,...,0.0,NaN,0.3,19.71,Credit Card,1.0,NaN,Green,Street Hail Liveries,0 days 00:18:34
4,4,Jan,Verifone Inc.,2019-01-01 00:19:06,2019-01-01 00:39:43,True,Standard,82,NaN,1.0,...,0.0,NaN,0.3,19.30,Cash,1.0,NaN,Green,Street Hail Liveries,0 days 00:20:37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6044045,6044045,Dec,NaN,2019-12-31 23:26:00,2019-12-31 23:54:00,False,Unknown,35,130,NaN,...,0.0,NaN,0.3,51.90,NaN,NaN,NaN,Green,Street Hail Liveries,0 days 00:28:00
6044046,6044046,Dec,NaN,2019-12-31 23:02:00,2019-12-31 23:16:00,False,Unknown,76,61,NaN,...,0.0,NaN,0.3,26.36,NaN,NaN,NaN,Green,Street Hail Liveries,0 days 00:14:00
6044047,6044047,Dec,NaN,2019-12-31 23:24:00,2019-12-31 23:40:00,False,Unknown,185,247,NaN,...,0.0,NaN,0.3,30.32,NaN,NaN,NaN,Green,Street Hail Liveries,0 days 00:16:00
6044048,6044048,Dec,NaN,2019-12-31 23:16:00,2019-12-31 23:37:00,False,Unknown,51,126,NaN,...,0.0,NaN,0.3,39.50,NaN,NaN,NaN,Green,Street Hail Liveries,0 days 00:21:00


In [ ]:
fhv_19_df

,index,month,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag,trip_duration
0,0,Jan,B00001,2019-01-01 00:30:00,2019-01-01 02:51:55,NaN,NaN,False,0 days 02:21:55
1,1,Jan,B00001,2019-01-01 00:45:00,2019-01-01 00:54:49,NaN,NaN,False,0 days 00:09:49
2,2,Jan,B00001,2019-01-01 00:15:00,2019-01-01 00:54:52,NaN,NaN,False,0 days 00:39:52
3,3,Jan,B00008,2019-01-01 00:19:00,2019-01-01 00:39:00,NaN,NaN,False,0 days 00:20:00
4,4,Jan,B00008,2019-01-01 00:27:00,2019-01-01 00:37:00,NaN,NaN,False,0 days 00:10:00
...,...,...,...,...,...,...,...,...,...
23130805,23130805,Jan,B03157,2019-01-31 23:21:00,2019-01-31 23:28:52,NaN,NaN,False,0 days 00:07:52
23130806,23130806,Jan,B03157,2019-01-31 23:38:04,2019-01-31 23:45:43,NaN,NaN,False,0 days 00:07:39
23130807,23130807,Jan,B03157,2019-01-31 23:53:58,2019-02-01 00:25:04,NaN,NaN,False,0 days 00:31:06
23130808,23130808,Jan,B03157,2019-01-31 23:54:53,2019-02-01 00:00:14,NaN,NaN,False,0 days 00:05:21


In [ ]:
fhvhv_19_df

,index,month,company,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag,license_type,trip_duration
0,0,Feb,Uber,B02867,2019-02-01 00:05:18,2019-02-01 00:14:57,245,251,False,High-Volume For-Hire Services,0 days 00:09:39
1,1,Feb,Uber,B02879,2019-02-01 00:41:29,2019-02-01 00:49:39,216,197,False,High-Volume For-Hire Services,0 days 00:08:10
2,2,Feb,Lyft,B02510,2019-02-01 00:51:34,2019-02-01 01:28:29,NaN,234,False,High-Volume For-Hire Services,0 days 00:36:55
3,3,Feb,Lyft,B02510,2019-02-01 00:03:51,2019-02-01 00:07:16,87,87,False,High-Volume For-Hire Services,0 days 00:03:25
4,4,Feb,Lyft,B02510,2019-02-01 00:09:44,2019-02-01 00:39:56,87,198,False,High-Volume For-Hire Services,0 days 00:30:12
...,...,...,...,...,...,...,...,...,...,...,...
255877365,23130712,Jan,Juno,B02914,2019-01-31 23:09:12,2019-01-31 23:21:25,69,169,False,High-Volume For-Hire Services,0 days 00:12:13
255877366,23130713,Jan,Juno,B02914,2019-01-31 23:14:37,2019-01-31 23:21:25,243,244,False,High-Volume For-Hire Services,0 days 00:06:48
255877367,23130714,Jan,Juno,B02914,2019-01-31 23:20:41,2019-01-31 23:53:05,61,10,False,High-Volume For-Hire Services,0 days 00:32:24
255877368,23130715,Jan,Juno,B02914,2019-01-31 23:33:45,2019-01-31 23:45:16,163,75,False,High-Volume For-Hire Services,0 days 00:11:31


## Some quick validation

In [ ]:
for name, df in dfs.items():
    print('\n\n')
    print(name)
    print(df.info())

## Finally save all these cleanish versions of the data

In [ ]:
def df_to_feather(df, suffix):
    df.reset_index().drop(labels='trip_duration', axis=1, errors='ignore').to_feather(f'./Data/{get_name_of(df)}{suffix}.feather')

In [ ]:
def feather_all(with_suffix):
    for name, df in dfs.items():
        df_to_feather(df, with_suffix)

In [ ]:
def df_from_feather(filepath, name):
    df = pd.read_feather(filepath).drop('index', errors='ignore')
    df['trip_duration'] = df.dropoff_datetime - df.pickup_datetime
    dfs[name] = df
    return df

In [ ]:
feather_all("")

## Load in cleaned feather data

In [ ]:
dfs = dict()
%time green_19_df = df_from_feather("./Data/green_2019_ETL.feather", "Boro Taxi 2019")
%time yellow_19_df = df_from_feather("./Data/yellow_2019_ETL.feather", "Yellow Taxi 2019")
# %time fhv_19_df = df_from_feather("./Data/fhv_2019_ETL.feather", "fhv_2019")
%time fhvhv_19_df = df_from_feather("./Data/fhvhv_2019_ETL.feather", "For Hire Vehicle High Volume 2019")

CPU times: user 844 ms, sys: 1.1 s, total: 1.94 s
Wall time: 1.01 s
CPU times: user 11.3 s, sys: 14.4 s, total: 25.8 s
Wall time: 13.8 s
CPU times: user 35.8 s, sys: 13.8 s, total: 49.6 s
Wall time: 43.1 s


In [ ]:
df_properties = {
    'Yellow Taxi 2019': {'color': 'orange'},
    'Boro Taxi 2019': {'color': '#8CFF00'},
    'For Hire Vehicle 2019': {'color': '#000000'},
    'For Hire Vehicle High Volume 2019': {'color': '#000000'}
}
def get_df_with_name(name):
    return dfs[name]

def get_name_of(find_df):
    for name, df in dfs.items():
        if df is find_df:
            return name
    raise Exception('Sorry, this dataframe was not found in `dfs`!')
    
def get_color_for(df):
    return df_properties[get_name_of(df)]['color']

In [ ]:
ETL_FILENAMES = [
    'yellow_2019',
    'green_2019',
    'fhv_2019',
    'fhvhv_2019'
]

# Merge Frames

## Map base number to company

In [ ]:
company_base_df = (fhvhv_19_df[['company', "dispatching_base_num"]]
                   .drop_duplicates()
                   .set_index('dispatching_base_num')
                   .drop(index=np.nan))

# from trip_record_user_guide.pdf
company_base_df.index = company_base_df.index.astype('category').add_categories(['B02907', 'B02908'])
company_base_df.loc['B02907'] = 'Juno'
company_base_df.loc['B02908'] = 'Juno'

company_base_df.company = company_base_df.company.astype(COMPANY_CATS)
company_base_df = company_base_df.sort_values(by='company')

In [ ]:
company_base_df.reset_index().drop('index', axis=1, errors='ignore').to_feather('./Data/company_base.feather')

In [ ]:
company_base_df = pd.read_feather('./Data/company_base.feather').set_index('dispatching_base_num')

In [ ]:
company_base_df.company = company_base_df.company.astype(COMPANY_CATS)

## Merge Jan FHV with FHVHV

In [ ]:
fhv_jan_19_df = fhv_19_df.merge(company_base_df, left_on='dispatching_base_num', right_index=True)

if not (fhvhv_19_df.month == 'Jan').sum():
    fhvhv_19_df = fhvhv_19_df.append(fhv_jan_19_df)

fhvhv_19_df['license_type'] = 'High-Volume For-Hire Services'
fhvhv_19_df.license_type = fhvhv_19_df.license_type.astype(LICENSE_CATS)

In [ ]:
fhvhv_19_df.name = "fhvhv_2019"
df_to_feather(fhvhv_19_df, "incl_Jan")

## Feather once again

In [ ]:
feather_all("_ETL")

# Anomalies and outliers

## See overall patterns in the data with a parallel axes plot

In [ ]:
def parallel_coords_plot(df, category):
    
    MAX_SAMPLES = 300
    COLORS = (
        '#ff5722',
        '#0f4c75',
        '#c3aed6',
        '#6e6d6d',
        '#fbd46d',
        '#fa26a0',
        '#81b214',
        '#00b7c2'
    )

    if not isinstance(df[category].dtype, pd.CategoricalDtype):
        raise ValueError("Category key was not found to be a pd.CategoricalDtype")

    uniques = df[category].unique()
    
    scaler = preprocessing.MinMaxScaler()
    
    cats_to_sample = [val for val in uniques if (df[df[category] == val].shape[0] > MAX_SAMPLES)]
    
    unsampled_df = df[~df[category].isin(cats_to_sample)].sample(frac=1)
    
    resampled_df = pd.concat(
        [df[df[category] == cat].sample(n=MAX_SAMPLES) for cat in cats_to_sample],
        axis=0, 
        join='outer'
    )
    
    numeric_attributes = [attr for attr in df.columns 
                          if (is_numeric_dtype(df[attr]) 
                              and (np.nan not in df[attr].value_counts(normalize=True, dropna=False).index 
                                   or df[attr].value_counts(normalize=True, dropna=False)[np.nan] < 0.5
                                  )
                             )
                         ]
    
    numeric_unsampled_df = unsampled_df[numeric_attributes]
    numeric_resampled_df = resampled_df[numeric_attributes]
    
#     return numeric_unsampled_df, numeric_resampled_df, df[numeric_attributes]
    scaler.fit(df[numeric_attributes])
    if 0 not in numeric_unsampled_df.shape: 
        unsampled_df[numeric_attributes] = scaler.transform(numeric_unsampled_df)
    if 0 not in numeric_resampled_df.shape:
        resampled_df[numeric_attributes] = scaler.transform(numeric_resampled_df)
    
    fig, rates_pcp_ax = plt.subplots()

    rates_pcp_ax = pd.plotting.parallel_coordinates(pd.concat([unsampled_df, resampled_df]), 
                                                    category, 
                                                    color=COLORS,
                                                    cols=numeric_attributes,
                                                    ax=rates_pcp_ax
                                                   )

    labels = rates_pcp_ax.get_xticklabels()
    plt.setp(labels, rotation=45, horizontalalignment='right')
    rates_pcp_ax.set_title(f"{category.capitalize()} ({get_name_of(df)})", fontsize=30)
    rates_pcp_ax.set(xlabel='Attribute', ylabel='Normalised Value')
    rates_pcp_ax.legend(loc='center left', bbox_to_anchor=(1,1))
    fig.set_facecolor('white')

    fig.tight_layout()
    fig.savefig(f"./Figures/{category}_pcp_{get_name_of(df)}_{datetime.now()}.png", pad=1)

    return fig

In [ ]:
yellow_ratecode_pcp = parallel_coords_plot(yellow_19_df, 'ratecode')

In [ ]:
yellow_vendor_pcp = parallel_coords_plot(yellow_19_df, 'vendor')
yellow_vendor_pcp

In [ ]:
yellow_DOLocation_pcp = parallel_coords_plot(yellow_19_df, 'DOLocationID')
yellow_DOLocation_pcp

In [ ]:
yellow_PULocation_pcp = parallel_coords_plot(yellow_19_df, 'PULocationID')
yellow_PULocation_pcp

In [ ]:
yellow_payment_pcp = parallel_coords_plot(yellow_19_df, 'payment_type')
yellow_payment_pcp

In [ ]:
def plot_all_parallel_coords_for(df):
    plots = []
    categoricals = [attribute for attribute in yellow_19_df.columns if isinstance(df[attribute].dtype, CategoricalDtype)] 
    for category in categoricals:
        plots.append(parallel_coords_plot(df, category))
    return plots

## Graphs distributions to find outliers & anomalies

In [ ]:
yellow_19_df.passenger_count = yellow_19_df.passenger_count.fillna(0.0).astype('int')

### Plot the count distributions

In [ ]:
def plot_int_val_hist(dfs, int_attribute, log=False):
    
    fig, axes = plt.subplots(ncols=len(dfs))
    
    for i, df in enumerate(dfs):
    
        print(df[int_attribute].value_counts(normalize=True, dropna=False))
        
        max_val = np.max(df[int_attribute].fillna(0).astype('int').unique())
        axes[i-1].hist(df[int_attribute].fillna(0).astype('int'),
                       density=True, 
                       log=log, 
                       bins=range(max_val+2), 
                       align='left', 
                       rwidth=0.8,
                       color=get_color_for(df),
                       label=[str(i) for i in range(max_val)]
                      )
        
        attribute = int_attribute.replace('_', ' ').capitalize()
        axes[i-1].set_title(f"{attribute} distribution ({get_name_of(df)})", fontsize=15, pad=12)
        axes[i-1].set_xlabel(f'{attribute} per trip')
        axes[i-1].set_ylabel('proportion of trips')
        axes[i-1].set_xticks(range(max_val+1))
        axes[i-1].grid(which='major', axis='y')
    
    fig.set_figheight(8)
    fig.set_figheight(5)
    fig.tight_layout()
    fig.savefig(f"./Figures/{int_attribute}_dist_{datetime.now()}.png", pad=10)
    return fig

In [ ]:
passenger_dist = plot_int_val_hist([yellow_19_df, green_19_df], 'passenger_count', log=True)

## Plot categorical distributions to find irregularities

In [ ]:
sns.set(style='darkgrid')
def plot_count_cat(dfs, category, exp_scale=False, log_xform=False, normalize=False, vertical=False):
    
    fig, axes = plt.subplots()

    print("\n\n" + get_name_of(df))
    print(df[category].value_counts(normalize=normalize, dropna=False))

    categories = df[category].value_counts(normalize=normalize, dropna=False)
    category_index = categories.index
    category_counts = categories.apply(logify).values if log_xform else categories.values

    order = category_index.sort_values() if (is_numeric_dtype(category_index.dtype)
                                             or category_index.ordered
                                            ) else category_index

    width = len(category_index)*0.6 + 6

    sns.barplot(x=category_index.values, 
                y=categories,
                order=order,
                color=get_color_for(df),
                saturation=1,
                ax=ax)

    ax.grid(True, which='both', axis='y')

    scale = 'log' if exp_scale else 'linear'
    ax.set_yscale(scale)

    ax.minorticks_on()
    ax.tick_params(axis='y', 
                   which="both", 
                   left=True, 
                   pad=10, 
                   color='gray', 
                   direction='out', 
                   length=5, width=0.5)
    ax.yaxis.grid(True)       
#         ax.xaxis.set_minor_locator(ticker.AutoMinorLocator(8))
    ax.xaxis.set_minor_locator(ticker.LogLocator(base=10.0, numticks=3, subs='auto'))

    # labels 
    ax.set_title(f'{category.replace("_", " ").capitalize()} {get_name_of(df)}', fontsize=15)
    num_prop = 'proportion' if normalize else 'number'
    ax.set_ylabel(f"{num_prop} of trips")
    ax.set_xlabel(category.capitalize().replace('_', ' '))
    ax.set_xticklabels(order, rotation=45, ha='right')
    sns.despine(trim=False, left=False)


    fig.set_figheight(5)
    fig.set_figwidth(width)
    fig.tight_layout()
    fig.savefig(f"./Figures/{category}_dist_{datetime.now()}.png", pad=10)
    return fig

In [ ]:
# sns.set_style("ticks", {"xtick.minor.size":8, "ytick.minor.size":8})
sns.set(style='darkgrid')
def plot_bar_cat_counts(dfs, category, exp_scale=False, log_xform=False, normalize=False, vertical=False, verbose=False):
    
    if vertical: 
        ncols = 1
        nrows = len(dfs)
    else:
        ncols =len(dfs)
        nrows = 1
    
    fig, axes = plt.subplots(ncols=ncols, nrows=nrows, sharey=not vertical)
    
    for i, df in enumerate(dfs):
        
        ax = axes[i-1] if len(dfs) > 1 else axes
        
        if verbose:
            print("\n\n" + get_name_of(df))
            print(df[category].value_counts(normalize=normalize, dropna=False))
        
        categories = df[category].value_counts(normalize=normalize, dropna=False)
        category_index = categories.index
        category_counts = categories.apply(logify).values if log_xform else categories.values
        
        order = category_index.sort_values() if (is_numeric_dtype(category_index.dtype)
                                                 or category_index.ordered
                                                ) else category_index
        
        width = len(category_index)*0.6 + 6
        
        sns.barplot(x=category_index.values, 
                    y=categories,
                    order=order,
                    color=get_color_for(df),
                    saturation=1,
                    ax=ax)
        
        ax.grid(True, which='both', axis='y')

        scale = 'log' if exp_scale else 'linear'
        ax.set_yscale(scale)
        
        ax.minorticks_on()
        ax.tick_params(axis='y', 
                       which="both", 
                       left=True, 
                       pad=10, 
                       color='gray', 
                       direction='out', 
                       length=5, width=0.5)
        ax.yaxis.grid(True)       
#         ax.xaxis.set_minor_locator(ticker.AutoMinorLocator(8))
        ax.xaxis.set_minor_locator(ticker.LogLocator(base=10.0, numticks=3, subs='auto'))
 
        # labels 
        ax.set_title(f'{category.replace("_", " ").capitalize()} {get_name_of(df)}', fontsize=15, pad=12)
        num_prop = 'proportion' if normalize else 'number'
        ax.set_ylabel(f"{num_prop} of trips", labelpad=10)
        ax.set_xlabel(category.capitalize().replace('_', ' '))
        ax.set_xticklabels(order, rotation=45, ha='right')
        sns.despine(trim=False, left=False)

            
    fig.set_figheight(5)
    fig.set_figwidth(width)
    fig.tight_layout()
    fig.savefig(f"./Figures/{category}_dist_{datetime.now()}.png", pad=10)
    return fig

In [ ]:
def plot_bar_plots_for(dfs, normalize=False, all_cats=False, include=[], exclude=[], verbose=False):
    
    plot_dict = dict()
    
    common_attrs = pd.Index([])
    for i, df in enumerate(dfs):
        if i == 1:
            common_attrs = df.columns.drop_duplicates()
        else:
            common_attrs.intersection(df.columns).drop_duplicates()
    
    for attr in common_attrs:
        if ((attr not in exclude) 
            and ((all_cats and isinstance(df[attr].dtype, pd.CategoricalDtype)) 
                 or attr in include)): 
            category = attr
            plot_dict[category] = plot_bar_cat_counts(dfs, category, normalize=normalize, exp_scale=True, verbose=verbose)
            
    plt.close('all')
    return plot_dict

In [ ]:
plot_dict_yg_norm = plot_bar_plots_for([yellow_19_df, green_19_df], 
                                       normalize=True,
                                       include=[
                                           'congestion_surcharge',
                                           'mta_tax',
                                           'improvement_surcharge',
                                           'payment_type',
                                           'vendor'
                                       ])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:68: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "pad" which is no longer supported as of 3.3 and will become an error two minor releases later
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:68: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "pad" which is no longer supported as of 3.3 and will become an error two minor releases later
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:68: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "pad" which is no longer supported as of 3.3 and will become an error two minor releases later
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:68: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "pad" which is no longer supported as of 3.3 and will become an error two minor releases later
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:68: Matplot

In [ ]:
plot_dict_yg_counts = plot_bar_plots_for([yellow_19_df, green_19_df], 
                               normalize=False,
                               include=[
                                   'ratecode',
                                   'passenger_count'
                               ])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:68: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "pad" which is no longer supported as of 3.3 and will become an error two minor releases later
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:68: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "pad" which is no longer supported as of 3.3 and will become an error two minor releases later


In [ ]:
plot_dict_all = plot_bar_plots_for(
    [yellow_19_df, green_19_df, fhvhv_19_df],
    normalize = False,
    include = [
        'license_type',
        'company', 
        'month', 
        'SR_Flag', 
        'DOLocationID', 
        'PULocationID'
    ]
)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:68: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "pad" which is no longer supported as of 3.3 and will become an error two minor releases later
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:68: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "pad" which is no longer supported as of 3.3 and will become an error two minor releases later
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:68: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "pad" which is no longer supported as of 3.3 and will become an error two minor releases later
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:68: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "pad" which is no longer supported as of 3.3 and will become an error two minor releases later
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:68: Matplot

In [ ]:
yellow_19_df.info()

In [ ]:
nyc_taxi_df = pd.concat([yellow_19_df, green_19_df], axis=0).drop('index', axis=1)

In [ ]:
tlc_19.company = tlc_19.company.astype(COMPANY_CATS)

In [ ]:
COMPANY_COLOURS = {
    'Uber': '#000000',
    'Lyft': '#FF00BF',
    'Via': '#00B7E8',
    'Juno': '#234F9D',
    'Yellow': '#FEC63F',
    'Green': '#8CFF00'
}

LICENSE_COLOURS = {
    'High-Volume For-Hire Services': '#000000',
    'Street Hail Liveries': '#8CFF00',
    'Medallion': '#FEC63F'
}

In [ ]:
months_count_fig, months_count_ax = plt.subplots()
sns.countplot(x='month', hue='license_type', data=tlc_19, ax=months_count_ax, palette=LICENSE_COLOURS, saturation=1.0)

In [ ]:
months_count_ax.set_ylabel("Number of trips")
months_count_ax.set_title(f'Rides per month by license type 2019', fontsize='xx-large')
months_count_fig.tight_layout()

In [ ]:
months_count_fig.

## Plot distribution of continuous attributes to find irregularities

In [ ]:
df_yellow_19_feb.drop(df_yellow_19_feb[df_yellow_19_feb.fare_amount > df_yellow_19_feb.fare_amount.quantile(0.99999)].index, inplace=True)

def plot_continuous_violin_dists(df, all_continuous=False, include=[], exclude=[]):

In [ ]:
def plot_continuous_violin_dist(df, attribute, given=None):
    
    unique_categories = df[given].unique() if given else [None]
    
    fig, axes = plt.subplots(nrows=len(unique_categories) if given else 1, sharex=False)

    for i, category_value in enumerate(unique_categories):

        ax = axes[i-1] if len(unique_categories) >= 1 else axes
        df_given_category = df[df[given] == category_value]
        
        sns.violinplot(x=df_given_category[attribute], ax=ax, inner='quartile', color=get_color_for(df))
        ax.set_title(category_value.capitalize(), loc='left', y=0.5, x=-10)
        ax.set_xlabel(attribute.replace('_', ' ').capitalize())

    attribute_str = attribute.replace('_', ' ').capitalize()
    fig_title = (f'{attribute_str} distribution by {given} value' 
                 if given else f'{attribute_str} distribution')
    
    fig.suptitle(fig_title, fontsize=20)
    fig.set_figheight=len(unique_categories)*1
    fig.tight_layout()
    fig.savefig(f'./Figures/{fig_title}.png')
    
    return fig

In [ ]:
fare_amount_dist_fig = plot_continuous_violin_dist(yellow_19_df, 'fare_amount', given='ratecode')

In [ ]:
fare_amount_dist_fig.set_figheight(10)
fare_amount_dist_fig.set_figwidth(15)
fare_amount_dist_fig.tight_layout()

In [ ]:
fare_amount_dist_fig.savefig('./Figures/fare_amount_dist.png')

In [ ]:
nyc_taxi_df.license_type

In [ ]:
fare_amount_fig, ax = plt.subplots()
df = nyc_taxi_df
# df.license_type.cat.remove_categories('High-Volume For-Hire Services', inplace=True)
df.license_type.cat

In [ ]:
ax = sns.violinplot(x='ratecode', 
                    y='fare_amount', 
                    hue='license_type',
                    hue_order=['Medallion', 'Street Hail Liveries'],
                    data=df, 
                    split=True,
                    scale='width',
                    scale_hue=False,
                    palette=[get_color_for(yellow_19_df), get_color_for(green_19_df)],
                    inner='box', 
                    orient='v', 
                    ax=ax)

In [ ]:
fare_amount_fig.tight_layout()

In [ ]:
fare_amount_fig.axes[0].set_title("Fare amount by metered rate", fontsize='xx-large')

In [ ]:
fare_amount_fig.savefig('./Figures/Fare_amount_cleaned_dist.png', pad=4)

In [ ]:
fare_amount_yg_comparison_fig, fare_amount_yg_comparison_ax = plt.subplots()

# Cleaning

In [ ]:
def cleanse_fare(df):
    
    # drop top 0.001% quantile ~.>$200
    df.drop(index=df[df.fare_amount > df.fare_amount.quantile(0.9999)].index, inplace=True)

    negative_fare = df[(df.fare_amount < 0)].index
    df.drop(index=negative_fare, axis=0, inplace=True)
    
    bad_jfk = df[(df.ratecode == 'JFK')&(df.fare_amount < 52.0)&(df.trip_distance < 14)].index
    df.drop(index=bad_jfk, inplace=True)

    no_go = df[(df.fare_amount == 0)&(df.trip_distance == 0.0)].index
    df.drop(index=no_go, inplace=True)

    return df

In [ ]:
def cleanse_trip_distance(df):
    
    negative_dists = df[df.trip_distance < 0].index
    df.drop(index=negative_dists, inplace=True)
    
    too_high = df[df.trip_distance > df.trip_distance.quantile(0.99999)].index
    df.drop(index=too_high, inplace=True)
    
    return df

In [ ]:
def cleanse_trip_duration(df):
    
    index_names = df.index.names
    if None not in index_names:
        df = df.reset_index()
    
    df = df.drop(
        index=(
            df[
                (
                    ((df.trip_duration) > np.timedelta64(3, 'h')) 
                    | (df.trip_duration < np.timedelta64(0, 's'))
                )
            ].index
        )
    )
    
    if None not in index_names:
        df = df.set_index(index_names)
    
    return df

In [ ]:
%%time
for name in dfs.keys():
    if 'For Hire' not in name:
        print('\n\n' + name)
        %time dfs[name] = cleanse_fare(dfs[name])
        %time dfs[name] = cleanse_trip_distance(dfs[name])
        %time dfs[name] = cleanse_trip_duration(dfs[name])



Boro Taxi 2019
CPU times: user 3.84 s, sys: 797 ms, total: 4.64 s
Wall time: 4.58 s
CPU times: user 1.85 s, sys: 428 ms, total: 2.28 s
Wall time: 2.26 s
CPU times: user 873 ms, sys: 217 ms, total: 1.09 s
Wall time: 1.09 s


Yellow Taxi 2019
CPU times: user 1min 28s, sys: 11.5 s, total: 1min 39s
Wall time: 1min 39s
CPU times: user 40.4 s, sys: 4.79 s, total: 45.1 s
Wall time: 44.9 s
CPU times: user 19.2 s, sys: 2 s, total: 21.2 s
Wall time: 21.2 s
CPU times: user 2min 34s, sys: 19.7 s, total: 2min 54s
Wall time: 2min 53s


In [ ]:
fhvhv_19_df = cleanse_trip_duration(fhvhv_19_df)
dfs['For Hire Vehicle High Volume 2019'] = fhvhv_19_df

In [ ]:
yellow_19_df = dfs['Yellow Taxi 2019']
green_19_df = dfs['Boro Taxi 2019']
fhvhv_19_df = dfs['For Hire Vehicle High Volume 2019']

In [ ]:
nyc_taxi_df = pd.concat([dfs['Yellow Taxi 2019'], dfs['Boro Taxi 2019']], axis=0).drop('index', axis=1)

dfs['NYC Taxi 2019'] = nyc_taxi_df

In [ ]:
tlc_19 = pd.concat([
    dfs['For Hire Vehicle High Volume 2019'].reset_index(),
    dfs['NYC Taxi 2019'].reset_index()
], axis=0, join='inner')
dfs['TLC 2019'] = tlc_19

In [ ]:
dfs.keys()

dict_keys(['Boro Taxi 2019', 'Yellow Taxi 2019', 'For Hire Vehicle High Volume 2019', 'NYC Taxi 2019', 'TLC 2019'])

In [ ]:
feather_all("_CLEANED")

In [ ]:
nyc_taxi_df = cleanse_fare(nyc_taxi_df)

In [ ]:
nyc_taxi_df = cleanse_trip_distance(nyc_taxi_df)

In [ ]:
fhvhv_19_df = cleanse_trip_duration(fhvhv_19_df)

In [ ]:
nyc_taxi_df = cleanse_trip_duration(nyc_taxi_df)

In [ ]:
tlc_19 = cleanse_trip_duration(tlc_19)

In [ ]:
tlc_19.trip_duration.describe()

count                    345733555
mean     0 days 00:17:55.163158644
std      0 days 00:13:14.871534124
min                0 days 00:00:00
25%                0 days 00:08:49
50%                0 days 00:14:24
75%                0 days 00:23:07
max                0 days 03:00:00
Name: trip_duration, dtype: object

In [ ]:
df_to_feather(tlc_19, "_CLEANSED")
df_to_feather(nyc_taxi_df, "_CLEANSED")

In [ ]:
nyc_taxi_df = df_from_feather("./Data/New York Taxi 2019_CLEANSED.feather", "NYC Yaxi 2019")

ValueError: Categorical categories must be unique

In [ ]:
tlc_19 = df_from_feather("./Data/TLC 2019_CLEANSED.feather", "TLC 2019")

ValueError: Categorical categories must be unique

In [ ]:
df_to_feather(tlc_19, "_CLEANSED")

# Stuffs

In [ ]:
trip_dist_fare_zero_df = nyc_taxi_df[(nyc_taxi_df.fare_amount == 0)&(nyc_taxi_df.trip_distance != 0)]

In [ ]:
sns.displot(x='trip_distance', data=trip_dist_fare_zero_df, kind='kde')

In [ ]:
nyc_taxi_df

In [ ]:
negative_fare = nyc_taxi_df[(nyc_taxi_df.fare_amount < 0)].index
negative_fare

In [ ]:
yellow_upper_quantiles = yellow_19_df.fare_amount.quantile([0.9, 0.99, 0.999, 0.9999, 0.99999, 0.999999])
yellow_upper_quantiles.to_csv('./Tables/yellow_fare_upper_q.csv')
yellow_upper_quantiles

In [ ]:
green_upper_quantiles = green_19_df.fare_amount.quantile([0.9, 0.99, 0.999, 0.9999, 0.99999, 0.999999])
green_upper_quantiles.to_csv('./Tables/green_fare_upper_q.csv')
green_upper_quantiles

In [ ]:
yellow_summary = yellow_19_df.fare_amount.describe()
yellow_summary.to_csv('./Tables/yellow_fare_summary.csv')
yellow_summary

In [ ]:
green_summary = green_19_df.fare_amount.describe()
green_summary.to_csv('./Tables/green_fare_summary.csv')
green_summary

In [ ]:
nyc_taxi_df.info()

In [ ]:
fig, ax = plt.subplots()
numeric = [attr for attr in nyc_taxi_df.columns if is_numeric_dtype(nyc_taxi_df[attr].dtype)]
sns.heatmap(nyc_taxi_df[numeric].corr(), ax=ax, square=True, center=0.0, cmap='PRGn')

In [ ]:
negative_fare_portion_df = nyc_taxi_df[nyc_taxi_df.fare_amount < 0]

In [ ]:
neg_fare_corr_fig, ax = plt.subplots()
sns.heatmap(negative_fare_portion_df[numeric].corr(), ax=ax, square=True, center=0.0, cmap='PRGn')

In [ ]:
neg_fare_payment_fig, ax = plt.subplots()
sns.countplot(x='payment_type', data=negative_fare_portion_df, ax=ax, palette='pastel')
ax.set(
    xlabel='Payment type', ylabel='Number of trips'
)
plt.xticks(rotation=90)
ax.set_title('Payment type distribution of negative fares', fontsize='x-large')
neg_fare_payment_fig.set_figwidth(5)
neg_fare_payment_fig.set_figheight(4)
neg_fare_payment_fig.tight_layout()
neg_fare_payment_fig.savefig('./Figures/neg_fare_payment.png', pad=1)

In [ ]:
negative_fare_portion_df.shape[0]/nyc_taxi_df.shape[0]

In [ ]:
jfk_trips_df = nyc_taxi_df[nyc_taxi_df.ratecode == 'JFK']

In [ ]:
jfk_trips_df[(jfk_trips_df.company=='Yellow') & (jfk_trips_df.fare_amount==0)].sort_values(by='trip_duration').payment_type.value_counts()

In [ ]:
jfk_less_than_52 = nyc_taxi_df[(nyc_taxi_df.ratecode == 'JFK')&(nyc_taxi_df.fare_amount < 52)&(nyc_taxi_df.fare_amount > 60)]

In [ ]:
(jfk_less_than_52.trip_distance < 14).sum() / jfk_less_than_52.shape[0]

In [ ]:
jfk_distance_fig, ax = plt.subplot()
sns.displot(data=jfk_less_than_52, x='trip_distance', kind='kde', ax=ax)
ax.set_xlabel("Trip Distance")
ax.set_title("Distance ")

In [ ]:
sns.distplot(a=jfk_less_than_52['trip_distance'], kde=True)

In [ ]:
no_charge_movement_df = nyc_taxi_df[(nyc_taxi_df.fare_amount == 0)&(nyc_taxi_df.trip_distance == 0.0)]

In [ ]:
jfk_bad_fares = df_yellow_19_feb[(df_yellow_19_feb.Ratecode == 'JFK')&(df_yellow_19_feb.fare_amount != 52.0)]

neg52 = jfk_bad_fares[jfk_bad_fares.fare_amount == -52.0]
neg52group = neg52.groupby('payment_type').agg(count=('payment_type', 'count')).reset_index()
neg52group

to_drop = df_yellow_19_feb[(df_yellow_19_feb.Ratecode == 'JFK')&(df_yellow_19_feb.fare_amount == -52.0)].index

df_yellow_19_feb.drop(to_drop, inplace=True)

no_go = df_yellow_19_feb[(df_yellow_19_feb.fare_amount == 0)&(df_yellow_19_feb.trip_distance == 0.0)].index
df_yellow_19_feb.drop(no_go, inplace=True)

negative_fare = df_yellow_19_feb[(df_yellow_19_feb.fare_amount < 0)].index
df_yellow_19_feb.drop(negative_fare, inplace=True)

df_yellow_19_feb['trip_duration'] = df_yellow_19_feb.tpep_dropoff_datetime - df_yellow_19_feb.tpep_pickup_datetime

drop2_rate_fares_fig, drop2_rate_fares_axs = plt.subplots(nrows=len(df_yellow_19_feb.Ratecode.unique()), sharex=True, sharey=False)

for i, ratecode in enumerate(df_yellow_19_feb.Ratecode.unique()):
    
    df = df_yellow_19_feb[df_yellow_19_feb.Ratecode == ratecode]
    sns.violinplot(x=df.fare_amount, ax=drop2_rate_fares_axs[i], inner='quartile', color='orange')
    drop2_rate_fares_axs[i].set_title(ratecode.capitalize(), loc='right', y=0.5)
    drop2_rate_fares_axs[i].set_xlabel('Fare amount $')

    
drop2_rate_fares_fig.suptitle("Ratecode Fare Amount", fontsize=20)
drop2_rate_fares_fig.savefig('ratecode_fare_violin_drop2.png')

## Trip Distance


In [ ]:
fhvhv_19_df

In [ ]:
yellow_19_df.info()

In [ ]:
sns.violinplot(y='trip_distance', hue='license_type', data=nyc_taxi_df)

In [ ]:
plt.hist()

In [ ]:
nyc_taxi_df_df.trip_distance.describe()

In [ ]:
nyc_taxi_df.trip_distance.quantile(0.99999)

In [ ]:
too_fast_too_far_df = nyc_taxi_df[(nyc_taxi_df.trip_distance > 100)&(nyc_taxi_df.trip_duration < np.timedelta64(1, 'h'))]

In [ ]:
too_fast_too_far_df.pickup_datetime.dt.time.sort_values()

In [ ]:
sns.distplot(too_fast_too_far_df['pickup_datetime'].dt.time, kde=True)

In [ ]:
sns.violinplot(x='', y='', hue=)

In [ ]:
sns.countplot(x='ratecode', data=too_fast_too_far_df)

In [ ]:
trip_dist_fig, trip_dist_ax = plt.subplots()
sns.distplot(df_yellow_19_feb.trip_distance.apply(logify), hist=False, rug=False, ax=trip_dist_ax)
trip_dist_ax.set(title='Trip distance kernel density estimate', xlabel='Trip distance', ylabel='Density')
trip_dist_fig.tight_layout()
trip_dist_fig.savefig('trip_KDE.png', pad=1)

In [ ]:
df_yellow_19_feb.trip_distance.describe()

In [ ]:
df_yellow_19_feb.trip_distance.quantile([0.9, 0.99, 0.999, 0.9999, 0.99999, 0.999999])

In [ ]:
plt.hist(df_yellow_19_feb.trip_distance, bins=[0,1,2,3,4,5,10,50,100,200], log=True)

In [ ]:
too_far = df_yellow_19_feb[(df_yellow_19_feb.trip_distance > 100)&(df_yellow_19_feb.trip_duration < np.timedelta64(1, 'h'))].index
df_yellow_19_feb.drop(too_far, inplace=True)

## Check that passenger counts make sense: MAY WANT TO REDISTRIBUTE 0

In [ ]:
figure, axes = plt.subplots()
plt.grid(which='major', axis='y')
max_passengers = df_yellow_19_feb.passenger_count.unique().max()
counts, bins, patches = plt.hist(df_yellow_19_feb.passenger_count, 
                                 density=True, 
                                 log=True, 
                                 bins=range(max_passengers+2), 
                                 align='left', 
                                 rwidth=0.8,
                                 color='orange',
                                 label=[str(i) for i in range(max_passengers)])
plt.title("Yellow taxi passenger count", fontsize=15)
plt.xlabel('# passengers / trip')
plt.ylabel('proportion of trips')
plt.xticks(range(max_passengers+1))
plt.show()

Now perform imputation on 0's as this is a driver entered value. Use KNN clustering to find the most appropriate count. 

In [ ]:
from sklearn.impute import KNNImputer, MissingIndicator
indicator = MissingIndicator(missing_values=0)


In [ ]:
df_yellow_19_feb.info()

In [ ]:
imputer = KNNImputer(n)
df_yellow_19_feb.

# Programmatically cleanse all

Name the dfs

Clean yellow

In [ ]:
for month in MONTHS:
    df = yellow19[month]
    print("Cleaning " + month)
    %time yellow19[month] = cleanse_trip_distance(cleanse_fare(df))
    print("Writing .feather...")
    %time yellow19[month].drop(labels='trip_duration', axis=1).reset_index().to_feather(f'./Data/Cleansed/{df.name}.feather'.replace(' ', '_'))

In [ ]:
for month in MONTHS:
    df = green19[month]
    print("Cleaning " + month)
    %time green19[month] = cleanse_trip_distance(cleanse_fare(df))
    print("Writing .feather...")
    %time green19[month].reset_index().drop(labels='trip_duration', axis=1).to_feather(f'./Data/Cleansed/{df.name}.feather'.replace(' ', '_'))

In [ ]:
# load in cleansed yellow 2019
for month in MONTHS:
    capmonth = month.capitalize()
    yellow19[month] = pd.read_feather(f'./Data/Cleansed/Yellow_Taxi_{capmonth}_2019.feather')

In [ ]:
# load in cleansed yellow 2019
for month in MONTHS:
    capmonth = month.capitalize()
    green19[month] = pd.read_feather(f'./Data/Cleansed/Green_Taxi_{capmonth}_2019.feather')

In [ ]:
df_to_feather(nyc_taxi_df, '_CLEANSED')


In [ ]:
nyc_taxi_df = df_from_feather("./Data/New York Taxi 2019_CLEANSED.feather", "nyc_taxi_2019")

# Find interesting features by correlation

In [ ]:
fhvhv_corr = fhvhv_19_df.corr()

In [ ]:
corr_fig, corr_axes = plt.subplots(ncols=3)
for i, corr_df in enumerate([yellow_corr, green_corr, fhvhv_corr]):
    ax = corr_axes[i-1]
    sns.heatmap()

# Reindex

In [ ]:
nyc_taxi_df

,month,vendor,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,ratecode,store_and_fwd_flag,PULocationID,DOLocationID,...,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,company,license_type,trip_duration,ehail_fee,trip_type
0,Jan,"Creative Mobile Technologies, LLC.",2019-01-01 00:46:40,2019-01-01 00:53:20,1.0,1.50,Standard,True,151,239,...,1.65,0.0,0.3,9.95,NaN,Yellow,Medallion,0 days 00:06:40,NaN,NaN
1,Jan,"Creative Mobile Technologies, LLC.",2019-01-01 00:59:47,2019-01-01 01:18:59,1.0,2.60,Standard,True,239,246,...,1.00,0.0,0.3,16.30,NaN,Yellow,Medallion,0 days 00:19:12,NaN,NaN
2,Jan,Verifone Inc.,2018-12-21 13:48:30,2018-12-21 13:52:40,3.0,0.00,Standard,True,236,236,...,0.00,0.0,0.3,5.80,NaN,Yellow,Medallion,0 days 00:04:10,NaN,NaN
3,Jan,Verifone Inc.,2018-11-28 15:52:25,2018-11-28 15:55:45,5.0,0.00,Standard,True,193,193,...,0.00,0.0,0.3,7.55,NaN,Yellow,Medallion,0 days 00:03:20,NaN,NaN
4,Jan,Verifone Inc.,2018-11-28 15:56:57,2018-11-28 15:58:33,5.0,0.00,JFK,True,193,193,...,0.00,0.0,0.3,55.55,NaN,Yellow,Medallion,0 days 00:01:36,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6044044,Dec,NaN,2019-12-31 23:46:00,2020-01-01 00:03:00,NaN,7.43,Unknown,False,7,56,...,0.00,0.0,0.3,24.50,NaN,Green,Street Hail Liveries,0 days 00:17:00,NaN,NaN
6044045,Dec,NaN,2019-12-31 23:26:00,2019-12-31 23:54:00,NaN,10.02,Unknown,False,35,130,...,0.00,0.0,0.3,51.90,NaN,Green,Street Hail Liveries,0 days 00:28:00,NaN,NaN
6044046,Dec,NaN,2019-12-31 23:02:00,2019-12-31 23:16:00,NaN,2.27,Unknown,False,76,61,...,0.00,0.0,0.3,26.36,NaN,Green,Street Hail Liveries,0 days 00:14:00,NaN,NaN
6044047,Dec,NaN,2019-12-31 23:24:00,2019-12-31 23:40:00,NaN,9.35,Unknown,False,185,247,...,0.00,0.0,0.3,30.32,NaN,Green,Street Hail Liveries,0 days 00:16:00,NaN,NaN


In [ ]:
tlc_19

,index,month,company,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,license_type,trip_duration
0,0,Feb,Uber,2019-02-01 00:05:18,2019-02-01 00:14:57,245,251,High-Volume For-Hire Services,0 days 00:09:39
1,1,Feb,Uber,2019-02-01 00:41:29,2019-02-01 00:49:39,216,197,High-Volume For-Hire Services,0 days 00:08:10
2,2,Feb,Lyft,2019-02-01 00:51:34,2019-02-01 01:28:29,NaN,234,High-Volume For-Hire Services,0 days 00:36:55
3,3,Feb,Lyft,2019-02-01 00:03:51,2019-02-01 00:07:16,87,87,High-Volume For-Hire Services,0 days 00:03:25
4,4,Feb,Lyft,2019-02-01 00:09:44,2019-02-01 00:39:56,87,198,High-Volume For-Hire Services,0 days 00:30:12
...,...,...,...,...,...,...,...,...,...
89939642,6044044,Dec,Green,2019-12-31 23:46:00,2020-01-01 00:03:00,7,56,Street Hail Liveries,0 days 00:17:00
89939643,6044045,Dec,Green,2019-12-31 23:26:00,2019-12-31 23:54:00,35,130,Street Hail Liveries,0 days 00:28:00
89939644,6044046,Dec,Green,2019-12-31 23:02:00,2019-12-31 23:16:00,76,61,Street Hail Liveries,0 days 00:14:00
89939645,6044047,Dec,Green,2019-12-31 23:24:00,2019-12-31 23:40:00,185,247,Street Hail Liveries,0 days 00:16:00


In [ ]:
fhvhv_19_df = fhvhv_19_df.rename_axis(index=['idx']).reset_index().set_index(['month', 'company', 'idx'])

In [ ]:
yellow_19_df.rename_axis(index=['idx']).reset_index().set_index(['month', 'idx'])

In [ ]:
green_19_df = green_19_df.rename_axis(index=['idx']).reset_index().set_index(['month', 'idx'])

In [ ]:
nyc_taxi_df = nyc_taxi_df.rename_axis(index=['idx']).reset_index().set_index(['month', 'idx'])

# Mapping

In [ ]:
zone_polgns_gdf = gpd.read_file("./Data/taxi_zones/taxi_zones.shp")
zone_polgns_gdf.geometry = zone_polgns_gdf.geometry.to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
zone_polgns_gdf.set_index('LocationID', drop=False, inplace=True)
zone_polgns_gdf.drop('OBJECTID', axis=1, errors='ignore', inplace=True)
# zone_polgns_gdf['centroid'] = zone_polgns_gdf.geometry.centroid

In [ ]:
zone_polgns_gdf

,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
LocationID,,,,,,
1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((-74.18445 40.69500, -74.18449 40.695..."
2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ..."
3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((-73.84793 40.87134, -73.84725 40.870..."
4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((-73.97177 40.72582, -73.97179 40.725..."
5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((-74.17422 40.56257, -74.17349 40.562..."
...,...,...,...,...,...,...
259,0.126750,0.000395,Woodlawn/Wakefield,259,Bronx,"POLYGON ((-73.85107 40.91037, -73.85207 40.909..."
260,0.133514,0.000422,Woodside,260,Queens,"POLYGON ((-73.90175 40.76078, -73.90147 40.759..."
261,0.027120,0.000034,World Trade Center,261,Manhattan,"POLYGON ((-74.01333 40.70503, -74.01327 40.704..."


## Mean fare /zone

In [ ]:
zones_geo_JSON = zone_polgns_gdf[['LocationID', 'geometry']].drop_duplicates('LocationID').to_json()

In [ ]:
zone_mean_fare_df = yellow_19_df.loc['Jan'].groupby('PULocationID').agg(avg_fare=('fare_amount', 'mean'))
zone_mean_fare_df = zone_mean_fare_df.reset_index()
zone_mean_fare_df

In [ ]:
nyc_yellow_zone_avg_fare_gdf = (
    gpd.GeoDataFrame(
        pd.merge(
            zone_mean_fare_df, zone_polgns_gdf, left_on='PULocationID', right_on='LocationID' )
    ).drop('LocationID', axis=1)
)
nyc_yellow_zone_avg_fare_gdf['log_avg_fare'] = nyc_yellow_zone_avg_fare_gdf['avg_fare'].apply(logify)
nyc_yellow_zone_avg_fare_gdf

Create a new folium Map

In [ ]:
nyc = folium.Map(location=(40.66, -73.94), tiles="Stamen Terrain", zoom_start=10)
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(nyc)

In [ ]:
folium.Choropleth(
    geo_data=zones_geo_JSON,
    name='Average Fare',
    data=nyc_yellow_zone_avg_fare_gdf,
    columns=['PULocationID', 'avg_fare'],
    key_on='feature.properties.LocationID',
    legend_name='Average fare by pickup zone',
    fill_opacity=0.8,
    line_opacity=0.1,
    highlight=True
).add_to(nyc)

style_func = lambda x: {
    'fillColor': '#ffffff',
    'color': '#000000',
    'fillOpacity': 0.1,
    'weight': 0.1
}

ZoneName = folium.features.GeoJson(
    nyc_yellow_zone_avg_fare_gdf,
    style_function=style_func,
    control=False, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=[
            'zone', 
            'PULocationID',
            'borough',
            'avg_fare'
        ],
        aliases=[
            'Zone Name:',
            'LocationID:',
            'Borough:',
            'Average fare:'
        ],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;")
    )
)
nyc.add_child(ZoneName)
nyc.keep_in_front(ZoneName)
folium.LayerControl().add_to(nyc)

# Market dominance

In [ ]:
# tlc_19 = pd.concat([
#     fhvhv_19_df.reset_index(),
#     nyc_taxi_df
# ], axis=0, join='inner')

tlc_19.company = tlc_19.company.astype(COMPANY_CATS)

In [ ]:
tlc_19 = tlc_19.rename_axis(index=['idx'])
tlc_19 = tlc_19.set_index(['month', 'idx'])

KeyError: "None of ['idx'] are in the columns"

In [ ]:
num_pickups_by_location_company_df = tlc_19.reset_index().groupby(['PULocationID', 'company']).agg(trip_count=('company', 'count'))

zone_pickup_dominant_company_df = (num_pickups_by_location_company_df
                                  .iloc[
                                      num_pickups_by_location_company_df
                                      .reset_index()
                                      .reset_index()
                                      .groupby(['PULocationID'])
                                      .trip_count
                                      .idxmax()
                                      .values
                                  ]
                                 )
zone_pickup_dominant_company_df = (zone_pickup_dominant_company_df
                                   .reset_index()
                                   .drop('index', axis=1, errors='ignore')
                                   .set_index(['PULocationID']))
zone_pickup_dominant_company_df.index = zone_pickup_dominant_company_df.index.rename('LocationID')

zone_pickup_dominant_company_gdf = (
    gpd.GeoDataFrame(
        zone_pickup_dominant_company_df.merge(zone_polgns_gdf, on='LocationID')
    )
)

zone_pickup_dominant_company_gdf = zone_pickup_dominant_company_gdf.reset_index().set_index('LocationID')

In [ ]:
zone_pickup_dominant_company_gdf.reset_index(inplace=True)

In [ ]:
zone_pickup_dominant_company_gdf.set_index('LocationID', drop=False, inplace=True)

In [ ]:
marketshare_2019 = folium.Map(location=(40.66, -73.94), tiles="Stamen Terrain", zoom_start=10)
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(marketshare_2019)

uber_colorscale = LinearColormap(
    ['#000000', '#323232', '#646464', '#959595', '#C7C7C7'],
    vmin=0, vmax=4000000
) 
lyft_colorscale = LinearColormap(
    ['#521180', '#7D0D90', '#A909A0', '#D404AF', '#FF00BF'], 
    vmin=0, vmax=4000000
)
via_colorscale = LinearColormap(
    ['#00316E', '#00538D', '#0074AB', '#0096CA', '#00B7E8'], 
    vmin=0, vmax=4000000
)
yellow_taxi_colorscale = LinearColormap(
    ['#B52700', '#C74F10', '#DA7720', '#EC9E2F', '#FEC63F'], 
    vmin=0, vmax=4000000
)
green_apple_colorscale= LinearColormap(
    ['#004A13', '#23770E', '#46A50A', '#69D205', '#8CFF00'], 
    vmin=0, vmax=4000000
)

colorscales = {
    'Uber': uber_colorscale,
    'Lyft': lyft_colorscale,
    'Via': via_colorscale,
    'Yellow': yellow_taxi_colorscale,
    'Green': green_apple_colorscale
}

for name, colorscale in colorscales.items():
    colorscale.caption = f'Number of trips by {name} taxi'
    colorscale.to_step(index=range(0, 4000000, 800000))
    colorscale.add_to(marketshare_2019)

def get_color(zone):
    try: 
        cell = company_agg_pickups_by_zone_df.loc[zone]
        company = cell.company
        value = cell.trip_count
        colorscale = colorscales[company]
        return colorscale.rgb_hex_str(value)
    except:
        return 'Red'

folium.GeoJson(
    data='./Data/Borough Boundaries.geojson',
    name='Borough Boundaries',
    control=False,
    style_function=lambda features: {
        'weight': 1.5,
        'opacity': 0.8,
        'color': 'yellow',
        'fillOpacity': 0
    }
).add_to(marketshare_2019)

choropleth = folium.GeoJson(
    data=zone_pickup_dominant_company_gdf.drop('centroid', axis=1, errors='ignore'),
    name='Market domination',
    style_function=lambda features: {
        'fillColor' : get_color(features['properties']['LocationID']),
        'fillOpacity' : 0.7,
        'color': 'grey',
        'weight': 0.5
    },
    control=True, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=[
            'zone', 
            'LocationID',
            'borough',
            'company',
            'trip_count'
        ],
        aliases=[
            'Zone Name:',
            'LocationID:',
            'Borough:',
            'Dominant player:',
            'Trip count:'
        ],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;")
    )
).add_to(marketshare_2019)

folium.GeoJson(
    data='./Data/Subway Lines.geojson',
    name='NY Subway Lines',
    style_function=lambda features: {
        'weight': 0.5,
        'color': 'red',
        'opacity': 0.5
    }
).add_to(marketshare_2019)

folium.LayerControl().add_to(marketshare_2019)

marketshare_2019.save("marketshare_2019.html")

# JFK Travel time

## JFK from anywhere

In [ ]:
tlc_to_JFK_df = tlc_19[tlc_19.DOLocationID==zone_polgns_gdf[zone_polgns_gdf.zone=="JFK Airport"].index.ravel()[0]]

tlc_to_JFK_df.trip_duration = tlc_to_JFK_df.trip_duration / np.timedelta64(1, 'm')

/home/student.unimelb.edu.au/daniely/.local/lib/python3.6/site-packages/pandas/core/generic.py:5165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
zone_to_JFK_mean_travel_df = tlc_to_JFK_df.groupby('PULocationID').agg(avg_duration=('trip_duration', 'mean'))

In [ ]:
zone_to_JFK_mean_travel_df = (zone_to_JFK_mean_travel_df
                              .reset_index()
                              .drop('index', axis=1, errors='ignore')
                              .rename(columns={'PULocationID': 'LocationID'}))

In [ ]:
zone_to_JFK_mean_travel_df

,LocationID,avg_duration
0,1,72.158854
1,2,23.045513
2,3,38.656785
3,4,48.177171
4,5,59.381418
...,...,...
251,252,30.292406
252,253,31.961969
253,254,43.450061
254,255,44.107937


In [ ]:
zone_to_JFK_mean_travel_gdf = (
    gpd.GeoDataFrame(
        zone_to_JFK_mean_travel_df.merge(zone_polgns_gdf, left_index=True, right_index=True)
    )
)

In [ ]:
zone_to_JFK_mean_travel_gdf

,avg_duration,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
1,72.158854,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((-74.18445 40.69500, -74.18449 40.695..."
2,23.045513,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ..."
3,38.656785,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((-73.84793 40.87134, -73.84725 40.870..."
4,48.177171,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((-73.97177 40.72582, -73.97179 40.725..."
5,59.381418,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((-74.17422 40.56257, -74.17349 40.562..."
...,...,...,...,...,...,...,...
252,30.292406,0.158004,0.001025,Whitestone,252,Queens,"POLYGON ((-73.82050 40.80101, -73.82040 40.800..."
253,31.961969,0.036051,0.000078,Willets Point,253,Queens,"POLYGON ((-73.83908 40.76526, -73.83790 40.764..."
254,43.450061,0.085886,0.000360,Williamsbridge/Olinville,254,Bronx,"POLYGON ((-73.85187 40.87906, -73.85138 40.878..."
255,44.107937,0.062384,0.000172,Williamsburg (North Side),255,Brooklyn,"POLYGON ((-73.96176 40.72523, -73.96154 40.725..."


In [ ]:
zone_to_JFK_mean_travel_df

,LocationID,avg_duration
0,1,72.158854
1,2,23.045513
2,3,38.656785
3,4,48.177171
4,5,59.381418
...,...,...
251,252,30.292406
252,253,31.961969
253,254,43.450061
254,255,44.107937


In [ ]:
JFK_travel_time_map = folium.Map(location=(40.66, -73.94), tiles="Stamen Terrain", zoom_start=10)
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(JFK_travel_time_map)


folium.GeoJson(
    data='./Data/Borough Boundaries.geojson',
    name='Borough Boundaries',
    control=False,
    style_function=lambda features: {
        'weight': 1.5,
        'opacity': 0.8,
        'color': 'yellow',
        'fillOpacity': 0
    }
).add_to(JFK_travel_time_map)

choropleth = folium.Choropleth(
    geo_data=zone_to_JFK_mean_travel_gdf,
    data=zone_to_JFK_mean_travel_df,
    name='Mean travel time to JFK',
    columns=['LocationID', 'avg_duration'],
    key_on='feature.properties.LocationID',
    
).add_to(JFK_travel_time_map)

folium.GeoJson(
    data='./Data/Subway Lines.geojson',
    name='NY Subway Lines',
    style_function=lambda features: {
        'weight': 0.5,
        'color': 'red',
        'opacity': 0.5
    }
).add_to(JFK_travel_time_map)

folium.LayerControl().add_to(JFK_travel_time_map)

JFK_travel_time_map.save("JFK_travel_time_map.html")

## JFK from Midtown

In [ ]:
tlc_midtown_to_JFK = (tlc_to_JFK_df[
    tlc_to_JFK_df.PULocationID.isin(
        zone_polgns_gdf[
            zone_polgns_gdf.zone.str.contains('Midtown')
        ].index
    )
])

NameError: name 'tlc_to_JFK_df' is not defined

In [ ]:
tlc_midtown_to_JFK.trip_duration.describe()

count    395795.000000
mean         54.579362
std          55.808584
min         -21.866667
25%          38.466667
50%          49.700000
75%          63.833333
max        1439.866667
Name: trip_duration, dtype: float64

In [ ]:
midtown_JFK_time_fig, midtown_JFK_time_ax = plt.subplots()
sns.displot(
    x='trip_duration',
    data=tlc_midtown_to_JFK,
)
ax.set_title("Travel time Midtown Manhattan to JFK Airport ", fontsize='xx-large')
ax.set_xlabel('Travel time')
midtown_JFK_time_fig

/home/student.unimelb.edu.au/daniely/.local/lib/python3.6/site-packages/seaborn/distributions.py:2163: UserWarning: `displot` is a figure-level function and does not accept the ax= paramter. You may wish to try histplot.
  warnings.warn(msg, UserWarning)


Text(0.5, 17.200000000000003, 'Travel time')